In [ ]:
!pip install librosa 
!pip install soundfile
!pip install openai-whisper
!pip install ffmpeg

In [ ]:
import mimetypes
import re
import torch
import whisper
from whisper.utils import get_writer

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("large-v2", device=device)

In [ ]:
def transcribe_audio(audio, model):
    """
    Использует Whisper для перевода аудио в формат текста.
    Возвращает полученный текст и, если требуется, скачивает файл нужного формата.

        Параметры:
            audio (str): Путь аудио файла, которое нужно обработать.
            model (Whisper): Скачанная модель Whisper.
            output_directory (str): Путь для сохранения файлов.

        Возвращаемое значение:
            text (str): Расшифрованный текст
    """

    text = ""
    mime_type, _ = mimetypes.guess_type(audio)
    is_wrong_format = False

    if re.match("audio/*", mime_type):

        result = model.transcribe(audio, language="ru")
        options = {
            "max_line_width": None,
            "max_line_count": None,
            "highlight_words": False
        }

        for i in range(len(result["segments"])):
            text += result["segments"][i]["text"]

    else:
        is_wrong_format = True

    return text, is_wrong_format

In [ ]:
import librosa
import soundfile as sf
import os 

# Загрузка звукового файла
audio_path = '/kaggle/input/neznaika-records/001.wav'
waveform, sample_rate = librosa.load(audio_path)
name_directory = 'phrases'

os.mkdir(name_directory)
path_replicas = os.path.join(os.getcwd(), name_directory)


# Разделение файла по фразам
intervals = librosa.effects.split(waveform, top_db=75)

# Запись всех файлов в отдельную директорию
for i, (start, end) in enumerate(intervals):
    phrase = waveform[start:end]

    file_path = os.path.join(path_replicas, f"speech_{i+1}.wav")
    sf.write(file_path, phrase, sample_rate)
del waveform, sample_rate, intervals

In [ ]:
# Создание txt файла в котором {путь до звукового файла}|{текст фразы в этом файе} с помощью whisper.
from tqdm import tqdm
import wave
import contextlib

name_txt = 'audio_text_filelist.txt'

files = os.listdir(path_replicas)

for f in tqdm(files):
    path_wav = os.path.join(path_replicas, f)
    with contextlib.closing(wave.open(path_wav,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
    if 3 <= duration <= 4:
        
        text, _ = transcribe_audio(path_wav, model)
        text = text.replace('\n', '')

        with open(name_txt, 'a') as file:
            file.write(path_wav + '|' + text + '\n')
del files

In [ ]:
#разделение текстового файла на train, test
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('/kaggle/working/audio_text_filelist.txt', sep='|', names=['path', 'text'])
train, test = train_test_split(df, test_size=0.2, random_state=42)

for i, row in train.iterrows():
    line = row.path + '|' + row.text.strip()
    with open('/kaggle/working/ljs_audio_text_train_filelist.txt', 'a') as file:
        file.write(line + '\n')

for i, row in test.iterrows():
    line = row.path + '|' + row.text.strip()
    with open('/kaggle/working/ljs_audio_text_val_filelist.txt', 'a') as file:
        file.write(line + '\n')
        
del df, train, test

In [ ]:
!apt-get install espeak -y

In [ ]:
import os
!git clone https://github.com/BogdanNV/Model_tts.git


In [ ]:
os.chdir('/kaggle/working/Model_tts')
!pip install -r requirements.txt

In [ ]:
os.chdir('/kaggle/working/Model_tts/monotonic_align')
#%mkdir monotonic_align
!python setup.py build_ext --inplace

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!python train.py -c configs/vits2_ljs_base.json -m models/test